Automação de Suporte ao Cliente com Múltiplos Agentes
Nesta lição, você aprenderá sobre os seis elementos-chave que ajudam a melhorar ainda mais o desempenho dos Agentes:

Interpretação de Papéis
Foco
Ferramentas
Cooperação
Diretrizes
Memória

In [ ]:
from crewai import Agent, Task, Crew
from dotenv import load_dotenv
from pathlib import Path
from llms import Gemini, Groq, GeminiModels, GroqModels

: 

In [ ]:
# Carrega variáveis de ambiente
dotenv_path = Path(__file__).parent.parent / 'config' / '.env'
load_dotenv(dotenv_path)

In [ ]:
gemini = Gemini(GeminiModels.GEMINI_1_5_PRO)
gemini_llm = gemini.create_instance()

Interpretação de Papéis, Foco e Cooperação

In [ ]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

Ao não definir allow_delegation=False, allow_delegation assume seu valor padrão de True.
Isso significa que o agente pode delegar seu trabalho a outro agente que esteja mais apto a realizar uma determinada tarefa.

In [ ]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

Interpretação de Papéis: Ambos os agentes receberam um papel, um objetivo e uma história de fundo.
Foco: Ambos os agentes foram incentivados a entrar no personagem dos papéis que estão interpretando.
Cooperação: O Agente de Garantia da Qualidade do Suporte pode delegar trabalho de volta ao Agente de Suporte, permitindo que esses agentes trabalhem juntos.


Ferramentas, Diretrizes e Memória
Ferramentas
Importar ferramentas do CrewAI

In [ ]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

Ferramentas Personalizadas Possíveis
Carregar dados do cliente
Acessar conversas anteriores
Carregar dados de um CRM
Verificar relatórios de bugs existentes
Verificar solicitações de funcionalidades existentes
Verificar tíquetes em andamento
... e mais
Algumas maneiras de usar as ferramentas CrewAI.
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
Instanciar uma ferramenta de raspagem de documentos.
A ferramenta irá raspar uma página (apenas 1 URL) da documentação do CrewAI.

In [ ]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

Diferentes Maneiras de Fornecer Ferramentas aos Agentes
Nível do Agente: O Agente pode usar a(s) Ferramenta(s) em qualquer Tarefa que realizar.
Nível da Tarefa: O Agente usará a(s) Ferramenta(s) apenas ao realizar essa Tarefa específica.
Nota: As Ferramentas da Tarefa substituem as Ferramentas do Agente.

Criando Tarefas
Você está passando a Ferramenta no Nível da Tarefa.

In [ ]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

quality_assurance_review não está utilizando nenhuma Ferramenta
Aqui, o Agente de Garantia de Qualidade apenas revisará o trabalho do Agente de Suporte.

In [ ]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)

Criando a Equipe
Memória
Configurar memory=True ao montar a equipe ativa a Memória.

In [ ]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)


Executando a Equipe
Nota: LLMs podem fornecer diferentes saídas para a mesma entrada, então o que você obtém pode ser diferente do que é mostrado no vídeo.

Diretrizes
Ao executar o comando abaixo, você pode ver que os agentes e as respostas estão dentro do escopo do que esperamos deles.

In [ ]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

In [ ]:
from IPython.display import Markdown
Markdown(result)